In [2]:
import numpy as np

# Simple Black Scholes 

Through a change of variables, the Black Scholes PDE can be solved via a constant coefficient heat equation.

Recall the solution for a European call price satisfies
$$\begin{gather}
    v_t + \frac 12 \sigma^2 x^2 v_{xx} + rxv_x  = rv \\
      v(T,x) = \max(x-K,0) \\
      0 \leq t \leq T \\
         0 \leq x.
\end{gather}$$
If we take
$$
  u(\tau,z) = e^{r\tau}v(\tau=T-t, z = \log\left(\tfrac xK\right) + (r-\tfrac 12\sigma^2)\tau)
$$
then $u$ satisfies
$$\begin{gather}
  u_\tau = \frac {\sigma^2}2 u_{zz} \\
  u(0,z) = K(e^z-1){\bf 1}_{(z>0)} \\
  0\leq\tau\leq T \\
  -\infty < z < \infty  
\end{gather}$$

In [86]:
r = 0.015
sig = 0.1
T = 1
S = 50
K = 50

n = 10 # time steps
m = 10 # space steps

a = 0.5*sig**2
L = S + 4*sig*T
dt = T/n
dx = L/m
gam = 0.5*a*dt/dx**2

def f(x):
    return K*(np.exp(np.fmax(0,x))-1)

def l(t):
    return 0*np.ones(len(t))

def r(t):
    return (L-K)*np.ones(len(t))

4.999999999999999


In [5]:
w=np.linspace(0,10,11)
np.fmax(2,w)

array([  2.,   2.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.])

In [87]:
# u(t,x)
xi = np.linspace(0,L,m+1)
ti = np.linspace(0,T,n+1)
u = np.zeros( (n+1, m+1) )

# Boundary conditions
u[0,:] = f(xi)
u[:,0] = l(ti)
u[:,-1] = r(ti)

# Pre-compute A and B, initialize c
A = np.diag(1+2*gam*np.ones(m-1)) + np.diag(-gam*np.ones(m-2) ,-1) + np.diag(-gam*np.ones(m-2), 1)
B = np.diag(1-2*gam*np.ones(m-1)) + np.diag(gam*np.ones(m-2) ,-1) + np.diag(gam*np.ones(m-2), 1)
c = np.zeros(m-1)

In [88]:
# Iteratively solve
for i in range(1,n+1):
    #print('i=%d'%i)
    c[0] = gam*(u[i-1,0] + u[i,0])
    c[-1] = gam*(u[i-1,-1] + u[i,-1])    
    u[i,1:-1] = np.linalg.solve(A,B.dot(u[i-1,1:-1])+c)

In [89]:
u

array([[ 0.        ,  0.01      ,  0.04      ,  0.09      ,  0.16      ,
         0.25      ,  0.36      ,  0.49      ,  0.64      ,  0.81      ,
         1.        ],
       [ 0.        ,  0.07950535,  0.15291178,  0.22890055,  0.31266944,
         0.40697222,  0.51266944,  0.62890055,  0.75291178,  0.87950535,
         1.        ],
       [ 0.        ,  0.08598624,  0.1793676 ,  0.27545776,  0.37307925,
         0.47224881,  0.57307925,  0.67545776,  0.7793676 ,  0.88598624,
         1.        ],
       [ 0.        ,  0.10002118,  0.19545424,  0.29139582,  0.38899369,
         0.48822637,  0.58899369,  0.69139582,  0.79545424,  0.90002118,
         1.        ],
       [ 0.        ,  0.0965927 ,  0.19708782,  0.29740114,  0.39725924,
         0.49713563,  0.59725924,  0.69740114,  0.79708782,  0.8965927 ,
         1.        ],
       [ 0.        ,  0.10131053,  0.1996622 ,  0.29871056,  0.39847601,
         0.49846653,  0.59847601,  0.69871056,  0.7996622 ,  0.90131053,
         1.   